In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.util import img_as_ubyte
from scipy.ndimage import gaussian_filter, median_filter
from skimage.restoration import denoise_nl_means, estimate_sigma

# Notebook-friendly plot display
%matplotlib inline

In [ ]:
# Set your file path to singular .tif file
tif_path = 'example-data/example.tif'
# Use the nth frame for testing and select fluorescence channel
frame_index = 0
channel_index = 1

# Load TIFF without modifying it
with tifffile.TiffFile(tif_path) as tif:
    arr = tif.asarray()

# Handle multi-channel stack or single frame
if arr.ndim == 4:
    if arr.shape[1] <= 4:  # shape: (frames, channels, H, W)
        fl_image = arr[frame_index, channel_index]
    else:  # shape: (channels, frames, H, W)
        fl_image = arr[channel_index, frame_index]
elif arr.ndim == 3:
    fl_image = arr
else:
    raise ValueError("Unexpected TIFF shape:", arr.shape)

# Ensure 512x512
h, w = fl_image.shape[-2:]
if (h, w) != (512, 512):
    raise ValueError(
        f"Expected a 512×512 frame, got {h}×{w}. "
        "Either resize/crop or change the model configuration."
    )

plt.imshow(fl_image, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Contrast stretch
p_low, p_high = np.percentile(fl_image, (1, 99))
contrast_stretched = exposure.rescale_intensity(fl_image, in_range=(p_low, p_high))

# Denoise (Non-local Means, fixed h; for light salt-and-pepper trimming)
denoised = denoise_nl_means(contrast_stretched, h=0.06, fast_mode=True)

# Apply gaussian filter to help with boundaries
denoised = gaussian_filter(denoised, sigma=1.0)

# Normalize for display
normalized = img_as_ubyte(denoised / np.max(denoised))

plt.imshow(normalized)
plt.axis('off')
plt.show()